# Top problems

## Importing libraries

In [1]:
from collections import Counter
import numpy as np
import pandas as pd
import re

## Loading raw files

In [2]:
answers_raw             = pd.read_excel('./data/odpovedi_202105081142.xlsx', sheet_name='Sheet0')
courses_raw             = pd.read_excel('./data/predmety_202105081148.xlsx', sheet_name='Sheet0')
english_lemma_cloud_raw = pd.read_csv('./data/EnglishLemmaCloud.csv', usecols=['ID','lemma','SENTIMENT'])
questionnaires_raw      = pd.read_excel('./data/dotazniky_202105081150.xlsx', sheet_name='Sheet0')
questions_raw           = pd.read_excel('./data/otazky_202105081147.xlsx', sheet_name='Sheet0')
sections_raw            = pd.read_excel('./data/dot_sekce_202105081151.xlsx', sheet_name='Sheet0')
workplaces_raw          = pd.read_excel('./data/ho_pracoviste_202105081150.xlsx', sheet_name='Sheet0')

## Creating working tables from raw files

In [3]:
answers             = answers_raw.copy()
courses             = courses_raw.copy()
english_lemma_cloud = english_lemma_cloud_raw
questionnaires      = questionnaires_raw.drop(columns=['VEDLEJSKA']).drop_duplicates() # dropping VEDLEJSKA, as it duplicates ID
questions           = questions_raw.copy()
sections            = sections_raw.copy()
workplaces          = workplaces_raw.copy()

## Merging working tables into one df + cleansing data

In [4]:
df = pd.merge(questionnaires, answers,             how='inner', left_on='ID',            right_on='DOTAZNIK', suffixes=['_questionnaire','_answer'])
df = pd.merge(df,             workplaces,          how='left',  left_on='HO_PRACOVISTE', right_on='ID',       suffixes=['','_workplace'])
df = pd.merge(df,             courses,             how='left',  left_on='PREDMET',       right_on='ID',       suffixes=['','_course'])
df = pd.merge(df,             english_lemma_cloud, how='left',  left_on='ID_answer',     right_on='ID',       suffixes=['','_lemma'])
df = pd.merge(df,             questions,           how='left',  left_on='OTAZKA',        right_on='ID',       suffixes=['','_question'])
df = pd.merge(df,             sections,            how='left',  left_on='DOT_SEKCE',     right_on='ID',       suffixes=['','_section'])

print(df.columns)

df.drop(inplace=True,columns=['PREDMET','HO_PRACOVISTE','PROGRAM_PRACOVISTE','FORMA','OBOR','UKONCENI','VYSLEDEK','DATUM_VYPLNENI','DOBA_VYPLNENI','ID_answer','DOTAZNIK','OTAZKA','PEDAGOG','TYP_AKCE','HODNOTA_CLOB','ID','HODN_OBDOBI','AKTIVNI','PRETIZENO','NAZEV','ZKRATKA','ID_course','GARANTUJICI_PRACOVISTE','ID_lemma','ID_question','DOT_SEKCE','OTAZKA_ANG','OTAZKA_TYP','PORADI','PER_TYP_AKCE','HO_PRACOVISTE_question','POVINNA_ODPOVED','ID_section','DOT_CAST','NAZEV_ANG','PORADI_section','PER_TYP_AKCE_section','PER_PEDAGOG_section'])
df.rename(inplace=True,columns={'ID_questionnaire':'id_questionnaire', 'PROGRAM':'program', 'ID_answer':'id_answer', 'HODNOTA_CISLO':'score', 'KOD':'course_code', 'NAZEV_course':'course', 'OBDOBI':'period', 'SENTIMENT':'sentiment' ,'OTAZKA_question':'question' ,'PER_PEDAGOG':'per_teacher' ,'NAZEV_section':'section'})
df['section'] = df['section'].str.replace(u'\xa0', u' ')
df['section'] = df['section'].str.replace('Jak na mě působil vyučující z následujících hledisek \(hodnocení 4 – nejlepší až 1 - nejhorší, N - nemohu objektivně zhodnotit\)', 'Jak na mě působil vyučující')

print(df.columns)

Index(['ID_questionnaire', 'PREDMET', 'HO_PRACOVISTE', 'PROGRAM',
       'PROGRAM_PRACOVISTE', 'FORMA', 'OBOR', 'UKONCENI', 'VYSLEDEK',
       'DATUM_VYPLNENI', 'DOBA_VYPLNENI', 'ID_answer', 'DOTAZNIK', 'OTAZKA',
       'PEDAGOG', 'TYP_AKCE', 'HODNOTA_CISLO', 'HODNOTA_CLOB', 'ID',
       'HODN_OBDOBI', 'AKTIVNI', 'PRETIZENO', 'NAZEV', 'ZKRATKA', 'ID_course',
       'KOD', 'NAZEV_course', 'GARANTUJICI_PRACOVISTE', 'OBDOBI', 'ID_lemma',
       'lemma', 'SENTIMENT', 'ID_question', 'DOT_SEKCE', 'OTAZKA_question',
       'OTAZKA_ANG', 'OTAZKA_TYP', 'PORADI', 'PER_PEDAGOG', 'PER_TYP_AKCE',
       'HO_PRACOVISTE_question', 'POVINNA_ODPOVED', 'ID_section', 'DOT_CAST',
       'NAZEV_section', 'NAZEV_ANG', 'PORADI_section', 'PER_TYP_AKCE_section',
       'PER_PEDAGOG_section'],
      dtype='object')
Index(['id_questionnaire', 'program', 'score', 'course_code', 'course',
       'period', 'lemma', 'sentiment', 'question', 'per_teacher', 'section'],
      dtype='object')


## Adding year_semester, so data can be sorted from newest to oldest

In [5]:
df[['semester','year']] = df['period'].str.split(' ',expand=True)
df['id_semester'] = np.where(df['semester']=='ZS','0','1')
df['year_semester'] = df['year'] + '_' + df['id_semester']
df.drop(inplace=True,columns=['year','id_semester'])

df.head()

,id_questionnaire,program,score,course_code,course,period,lemma,sentiment,question,per_teacher,section,semester,year_semester
0,357017,B-IB,NaN,4IT152,Informatika (v angličtině),ZS 2019/2020,NaN,NaN,Co a jak by se na výuce mohlo zlepšit (cvičení...,0,Celková spokojenost s předmětem a náměty na zl...,ZS,2019/2020_0
1,357017,B-IB,NaN,4IT152,Informatika (v angličtině),ZS 2019/2020,NaN,NaN,Látka probíraná v předmětu se NEPŘEKRÝVALA s j...,0,Překryv s jinými předměty,ZS,2019/2020_0
2,357017,B-IB,3.0,4IT152,Informatika (v angličtině),ZS 2019/2020,NaN,NaN,Celkově jsem s kvalitou předmětu spokojen/a (h...,0,Celková spokojenost s předmětem a náměty na zl...,ZS,2019/2020_0
3,357017,B-IB,4.0,4IT152,Informatika (v angličtině),ZS 2019/2020,NaN,NaN,Srozumitelnost výkladu a schopnost vysvětlit l...,1,Jak na mě působil vyučující,ZS,2019/2020_0
4,357017,B-IB,4.0,4IT152,Informatika (v angličtině),ZS 2019/2020,NaN,NaN,Připravenost přednášek/cvičení,1,Jak na mě působil vyučující,ZS,2019/2020_0


## Checking shapes after merging

In [6]:
print('answers_raw\t\t'           + str(answers_raw.shape))
print('english_lemma_cloud_raw\t' + str(english_lemma_cloud_raw.shape))
print('courses_raw\t\t'           + str(courses_raw.shape))
print('questionnaires_raw\t'      + str(questionnaires_raw.shape))
print('questions_raw\t\t'         + str(questions_raw.shape))
print('sections_raw\t\t'          + str(sections_raw.shape))
print('workplaces_raw\t\t'        + str(workplaces_raw.shape))
print('df\t\t\t'                  + str(df.shape))

len(answers_raw) == len(df)

answers_raw		(105410, 7)
english_lemma_cloud_raw	(6666, 3)
courses_raw		(6678, 5)
questionnaires_raw	(12706, 12)
questions_raw		(52, 10)
sections_raw		(14, 7)
workplaces_raw		(36, 6)
df			(105410, 13)


True

## Aggregating mean and count of score by course and year_semester and sorting each course from newest to oldest

In [7]:
df_grouped = df.groupby(['course_code', 'course', 'year_semester'])['score'].agg([('score_count', 'count'), ('score_mean', 'mean')]).reset_index().sort_values(by=['course_code', 'year_semester'], ascending=(True,False))
df_grouped['score_mean'] = df_grouped['score_mean'].round(2)

df_grouped.head()

,course_code,course,year_semester,score_count,score_mean
0,11F201,"Finanční teorie, politika a instituce",2020/2021_0,14,3.79
1,1BP210,Bankovnictví a finanční instituce,2019/2020_1,13,3.54
2,1BP403,Bankovnictví II,2019/2020_1,6,2.33
3,1BP430,Fundamentální akciová analýza,2020/2021_0,10,3.90
4,1BP431,"Technická analýza na akciových, měnových a kom...",2020/2021_0,10,3.80


## Creating variable for the newest semester in data

In [8]:
newest_semester = df_grouped['year_semester'].max()
newest_semester

'2020/2021_0'

## Courses, which occur only in the newest semester, are evaluated by their average score from worst to best

In [9]:
df_new_courses = df_grouped.drop_duplicates(keep=False, subset=['course'], inplace=False)
df_new_courses = df_new_courses[(df_new_courses['year_semester'] == newest_semester)].sort_values(by='score_mean')
df_new_courses['course_type'] = 'new'

In [10]:
df_new_courses_negative = df_new_courses[df_new_courses['score_mean'] < 3].copy()
df_new_courses_negative['sentiment_type'] = 'negative'

df_new_courses_positive = df_new_courses[df_new_courses['score_mean'] == 4].copy()
df_new_courses_positive['sentiment_type'] = 'positive'

In [11]:
df_new_courses_negative.head()

,course_code,course,year_semester,score_count,score_mean,course_type,sentiment_type
448,TVSUZ1,"Uznání zápočtu TV z jiné fakulty, VŠ 1",2020/2021_0,4,1.00,new,negative
437,TVSPOS,Posilování,2020/2021_0,8,1.00,new,negative
438,TVSSER,Šerm,2020/2021_0,4,1.00,new,negative
169,4IT111,Úvod do programování v jazyce Python,2020/2021_0,44,2.32,new,negative
303,4IT575,Softwarové architektury,2020/2021_0,212,2.58,new,negative


In [12]:
df_new_courses_positive.head()

,course_code,course,year_semester,score_count,score_mean,course_type,sentiment_type
154,4EK212,Kvantitativní management,2020/2021_0,14,4.0,new,positive
432,TVSMFO,Malý fotbal,2020/2021_0,8,4.0,new,positive
57,2PR204,Právo sociálního zabezpečení,2020/2021_0,6,4.0,new,positive


## Courses, which occur in the newest semester as well as in at least one other, are evaluated by the absolute diff between newest semester and the previous one (whenever it was) absolute diff is used, so drop from 2 to 1 has the same weight as drop from 4 to 3

In [13]:
df_existing_courses = df_grouped[df_grouped['course'].duplicated(keep=False)].copy()
df_existing_courses['rank'] = df_existing_courses.groupby(['course'])['year_semester'].cumcount()
df_existing_courses = df_existing_courses[df_existing_courses['rank'] < 2].drop(columns=['rank']) # keeping only last two semesters for each course
df_existing_courses['score_mean_previous'] = df_existing_courses.groupby('course')['score_mean'].shift(-1) # adding score from the previous semester to the following one
df_existing_courses = df_existing_courses[df_existing_courses['year_semester'] == newest_semester] # keeping only the newest (current) semester
df_existing_courses['score_mean_diff'] = (df_existing_courses['score_mean'] - df_existing_courses['score_mean_previous']).round(2)
df_existing_courses['course_type'] = 'existing'

In [14]:
df_existing_courses_negative = df_existing_courses[df_existing_courses['score_mean_diff'] < -0.5].copy().sort_values(by='score_mean_diff')
df_existing_courses_negative['sentiment_type'] = 'negative'

df_existing_courses_positive = df_existing_courses[df_existing_courses['score_mean_diff'] > 0.5].copy().sort_values(by='score_mean_diff', ascending=False)
df_existing_courses_positive['sentiment_type'] = 'positive'

In [15]:
df_existing_courses_negative.head()

,course_code,course,year_semester,score_count,score_mean,score_mean_previous,score_mean_diff,course_type,sentiment_type
429,TVSHOK,Hokej,2020/2021_0,4,1.75,3.67,-1.92,existing,negative
447,TVSTHA,Thai box,2020/2021_0,8,2.12,3.88,-1.76,existing,negative
59,2PR441,Právo počítačové a informačních a komunikačníc...,2020/2021_0,44,2.75,3.62,-0.87,existing,negative
411,5HP416,Jak uspět v médiích,2020/2021_0,10,2.70,3.56,-0.86,existing,negative
423,TVSFKT,Funkční kruhový trénink,2020/2021_0,42,3.14,3.96,-0.82,existing,negative


In [16]:
df_existing_courses_positive.head()

,course_code,course,year_semester,score_count,score_mean,score_mean_previous,score_mean_diff,course_type,sentiment_type
377,4SA526,Nová média a sociální sítě (v angličtině),2020/2021_0,30,3.40,1.33,2.07,existing,positive
358,4ME499,Multimediální projektový seminář pro VS,2020/2021_0,14,3.71,2.62,1.09,existing,positive
374,4SA525,Informace a média,2020/2021_0,49,3.63,2.67,0.96,existing,positive
83,3LG526,Logistické systémy,2020/2021_0,10,3.50,2.56,0.94,existing,positive
339,4ME301,Sémiotika multimédií,2020/2021_0,30,3.73,2.83,0.90,existing,positive


## Appending tables new positive, new negative, existing positive, existing negative courses

In [17]:
df_appended = pd.concat([df_new_courses_positive, df_new_courses_negative, df_existing_courses_positive, df_existing_courses_negative],sort=False).sort_values(by='score_count', ascending=False)
df_appended['rank'] = np.arange(df_appended.shape[0])+1


len(df_appended) == len(df_new_courses_positive) + len(df_new_courses_negative) + len(df_existing_courses_positive) + len(df_existing_courses_negative)

True

## Adding reasons, why the worst courses are the worst

In [18]:
for index, row in df_appended.iterrows():
    
    if row['sentiment_type'] == 'negative':
    
        # negative sections and their scores

        df_worst_sections = df[(df['course'] == row['course']) & (df['year_semester'] == newest_semester) & (df['section'] != 'Celková spokojenost s předmětem a náměty na zlepšení')].groupby(['section'])['score'].agg([('score_mean','mean')]).reset_index()
        df_worst_sections = df_worst_sections[df_worst_sections['score_mean'].notna()].sort_values(by='score_mean')

        section = df_worst_sections['section'].iloc[0]
        section_score = df_worst_sections['score_mean'].iloc[0].round(2)

        # most frequent negative comments

        lemma = df[(df['course'] == row['course']) & (df['sentiment'] == 'Negative') & (df['year_semester'] == newest_semester) & (df['lemma'].notna())]['lemma'].copy()
        lemma = lemma.str.split()
        lemma_frequency = Counter([item for sublist in lemma.tolist() for item in sublist]).most_common()
        lemma_frequency_string = str([x for x in lemma_frequency])
        words_with_frequency_list = re.sub("[^\w]", " ",  lemma_frequency_string).split()
        most_frequent_words_list = [x for x in words_with_frequency_list if not x.isdigit()][:20]
        most_frequent_words = ', '.join(map(str, most_frequent_words_list))
        
    elif row['sentiment_type'] == 'positive':
    
        # positive sections and their scores

        df_best_sections = df[(df['course'] == row['course']) & (df['year_semester'] == newest_semester) & (df['section'] != 'Celková spokojenost s předmětem a náměty na zlepšení')].groupby(['section'])['score'].agg([('score_mean','mean')]).reset_index()
        df_best_sections = df_best_sections[df_best_sections['score_mean'].notna()].sort_values(by='score_mean', ascending=False)

        section = df_best_sections['section'].iloc[0]
        section_score = df_best_sections['score_mean'].iloc[0].round(2)

        # most frequent positive comments

        lemma = df[(df['course'] == row['course']) & (df['sentiment'] == 'Positive') & (df['year_semester'] == newest_semester) & (df['lemma'].notna())]['lemma'].copy()
        lemma = lemma.str.split()
        lemma_frequency = Counter([item for sublist in lemma.tolist() for item in sublist]).most_common()
        lemma_frequency_string = str([x for x in lemma_frequency])
        words_with_frequency_list = re.sub("[^\w]", " ",  lemma_frequency_string).split()
        most_frequent_words_list = [x for x in words_with_frequency_list if not x.isdigit()][:20]
        most_frequent_words = ', '.join(map(str, most_frequent_words_list))
    
    # creating columns with the section, corresponding score and the most frequent comments
    
    df_appended.loc[index,'reason_name']  = section
    df_appended.loc[index,'reason_value'] = section_score
    df_appended.loc[index,'reason_text']  = most_frequent_words

In [19]:
for index, row in df_appended.iterrows():
    if row['course_type'] == 'new':
        if row['sentiment_type'] == 'negative':
            sentence = row['course'] + ' má nízké hodnocení (' + str(row['score_mean']) + ') kvůli sekci ' + row['reason_name'] + ' (hodnocení: ' + str(row['reason_value']) + '). Nejfrekventovanější slova: ' + row['reason_text']
        elif row['sentiment_type'] == 'positive':
            sentence = row['course'] + ' má vysoké hodnocení (' + str(row['score_mean']) + ') díky sekci ' + row['reason_name'] + ' (hodnocení: ' + str(row['reason_value']) + '). Nejfrekventovanější slova: ' + row['reason_text']

    elif row['course_type'] == 'existing':
        if row['sentiment_type'] == 'negative':
            sentence = row['course'] + ' má pokles hodnocení (' + str(row['score_mean_diff']) + ') kvůli sekci ' + row['reason_name'] + ' (hodnocení: ' + str(row['reason_value']) + '). Nejfrekventovanější slova: ' + row['reason_text']
        elif row['sentiment_type'] == 'positive':
            sentence = row['course'] + ' má nárůst hodnocení (' + str(row['score_mean_diff']) + ') díky sekci ' + row['reason_name'] + ' (hodnocení: ' + str(row['reason_value']) + '). Nejfrekventovanější slova: ' + row['reason_text']
    
    # creating column with the final sentence
    
    df_appended.loc[index,'sentence'] = sentence

## Testing results

In [20]:
df_appended.head()

,course_code,course,year_semester,score_count,score_mean,course_type,sentiment_type,score_mean_previous,score_mean_diff,rank,reason_name,reason_value,reason_text,sentence
303,4IT575,Softwarové architektury,2020/2021_0,212,2.58,new,negative,NaN,NaN,1,Obsah a náročnost předmětu,2.50,"semester, subject, clear, task, time, student,...",Softwarové architektury má nízké hodnocení (2....
285,4IT522,Užití MS Excelu v podnikové praxi (v angličtině),2020/2021_0,129,3.28,existing,negative,3.85,-0.57,2,Jak na mě působil vyučující,3.10,"student, English, connection, problem, good, c...",Užití MS Excelu v podnikové praxi (v angličtin...
265,4IT494,Řízení podnikové výkonnosti v nástrojích CPM a...,2020/2021_0,128,3.06,existing,negative,3.62,-0.56,3,Obsah a náročnost předmětu,2.87,"semester, work, Ready, doc, Marysky, clear, in...",Řízení podnikové výkonnosti v nástrojích CPM a...
374,4SA525,Informace a média,2020/2021_0,49,3.63,existing,positive,2.67,0.96,4,Zajištění výuky distanční formou,4.00,"test, interest, Student, presentation, siroke,...",Informace a média má nárůst hodnocení (0.96) d...
59,2PR441,Právo počítačové a informačních a komunikačníc...,2020/2021_0,44,2.75,existing,negative,3.62,-0.87,5,Zajištění výuky distanční formou,2.25,"information, form, guest, Chaotic, share, begi...",Právo počítačové a informačních a komunikačníc...


In [21]:
for index, row in df_appended.sort_values(by='score_count', ascending=False).iterrows():
    if row['course_type'] == 'new':
        if row['sentiment_type'] == 'negative':
            print(row['course'] + ' má nízké hodnocení (' + str(row['score_mean']) + ') kvůli sekci ' + row['reason_name'] + ' (hodnocení: ' + str(row['reason_value']) + '). Nejfrekventovanější slova: ' + row['reason_text'])
            print('\n')
        elif row['sentiment_type'] == 'positive':
            print(row['course'] + ' má vysoké hodnocení (' + str(row['score_mean']) + ') díky sekci ' + row['reason_name'] + ' (hodnocení: ' + str(row['reason_value']) + '). Nejfrekventovanější slova: ' + row['reason_text'])
            print('\n')
    elif row['course_type'] == 'existing':
        if row['sentiment_type'] == 'negative':
            print(row['course'] + ' má pokles hodnocení (' + str(row['score_mean_diff']) + ') kvůli sekci ' + row['reason_name'] + ' (hodnocení: ' + str(row['reason_value']) + '). Nejfrekventovanější slova: ' + row['reason_text'])
            print('\n')
        elif row['sentiment_type'] == 'positive':
            print(row['course'] + ' má nárůst hodnocení (' + str(row['score_mean_diff']) + ') díky sekci ' + row['reason_name'] + ' (hodnocení: ' + str(row['reason_value']) + '). Nejfrekventovanější slova: ' + row['reason_text'])
            print('\n')

Softwarové architektury má nízké hodnocení (2.58) kvůli sekci Obsah a náročnost předmětu (hodnocení: 2.5). Nejfrekventovanější slova: semester, subject, clear, task, time, student, change, assignment, trainer, current, explain, requirement, work, confused, Pecinovsky, teach, issue, design, evaluate, object


Užití MS Excelu v podnikové praxi (v angličtině) má pokles hodnocení (-0.57) kvůli sekci Jak na mě působil vyučující (hodnocení: 3.1). Nejfrekventovanější slova: student, English, connection, problem, good, cooperate, focus, basic, lot, time, miss, advance, content, class, business, orient, master, English, level, Clarity


Řízení podnikové výkonnosti v nástrojích CPM a její reporting má pokles hodnocení (-0.56) kvůli sekci Obsah a náročnost předmětu (hodnocení: 2.87). Nejfrekventovanější slova: semester, work, Ready, doc, Marysky, clear, information, hand, Guests, want, communication, Doc, Maryss, lack, explanation, program, main, part, absolutely, disastrous


Informace a média m

## Writing results to csv

In [22]:
df_appended.to_csv('./data/top_problems.csv', encoding='utf-8-sig', index=False)